In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
import sympy as sp
import pandas as pd
import pickle

# getcontext().prec = 70  

import sympy as sp


### Functions:

In [2]:
def commute_pauli_strings(pauli_string1, pauli_string2):
    # Defining commutation rules
    commutation_rules = {
        'II': True,
        'XX': True,
        'YY': True,
        'ZZ': True,
        'XY': False,
        'YX': False,
        'XZ': False,
        'ZX': False,
        'YZ': False,
        'ZY': False,
    }
    
    # Variable to keep track of the sign (1 for commutation, -1 for anti-commutation)
    sign = 1
    
    # Check if the two Pauli strings commute or anti-commute based on commutation rules
    for p1, p2 in zip(pauli_string1, pauli_string2):
        key = p1 + p2
        if key in commutation_rules:
            if not commutation_rules[key]:
                sign *= -1

    return sign == 1

# build the stabilizer matrix:
def generate_repetition_code_stabilizers(n):
    stabilizers = []
    for i in range(n - 1):
        stabilizer = 'I' * i + 'XX' + 'I' * (n - i - 2)
        stabilizers.append(stabilizer)
    return stabilizers


# build the logicals:
def generate_logical_operators(n):
    il_operator = 'I' * n
    xl_operator = 'X' + 'I' * (n - 1)
    zl_operator = 'Z' * n
    yl_operator = 'Y' + 'Z' * (n - 1)
    
    return {
        'IL': il_operator,
        'XL': xl_operator,
        'ZL': zl_operator,
        'YL': yl_operator
    }
    
    

### Rest of code:

In [3]:
# choose a repetition code [n,1,n]:
n=4
k=1
d=n



stabilizers = generate_repetition_code_stabilizers(n)
print(f"stabilizers: {stabilizers}")

logical_operators = generate_logical_operators(n)
print(f"logical_operators: {logical_operators}")

# Define the characters
chars = ['I', 'X', 'Y', 'Z']
length = len(stabilizers[0])  # Length of the Pauli strings

# Initialize a list to store commuting strings
commuting_strings = []

# Generate all possible strings of the same length
for i in range(4 ** length):
    string = ''
    for j in range(length):
        string += chars[(i // (4 ** j)) % 4]
        
    # Check if the generated string commutes with both Pauli strings
    detectable = 0
    for stabilizer in stabilizers:
        commute = commute_pauli_strings(string, stabilizer)
        if not commute:
            detectable = 1
    if detectable == 0:
        commuting_strings.append(string)
    

commutation_dict = {}
for string in commuting_strings:
    commutation_dict[string] = []
    for logical_type in logical_operators:
        if logical_type == 'IL':
            continue
    # for logical, logical_str in zip([XL, ZL, YL], ['XL', 'ZL', 'YL']):
        if commute_pauli_strings(string, logical_operators[logical_type]):
            commutation_dict[string].append(logical_type)
        if commutation_dict[string] == ['XL', 'ZL', 'YL']:
            commutation_dict[string] = ['IL']

# print(commutation_dict)

# Define the variables for probabilities
pI, pX, pY, pZ = sp.symbols('pI pX pY pZ')

# Initialize a dictionary to store the probabilities of characters
char_probabilities = {'I': pI, 'X': pX, 'Y': pY, 'Z': pZ}

# Function to compute the probability of a string
def compute_probability(string):
    probability = 1
    for char in string:
        probability *= char_probabilities[char]
    return probability

# for logical_name in ['IL', 'XL', 'ZL', 'YL']:
#     logical_string = [key for key, value in commutation_dict.items() if value == [logical_name]]
#     print(f"{logical_name}_strings = {logical_string}")
#     final_prob = sum(compute_probability(string) for string in logical_string)
#     final_prob = sp.simplify(final_prob)

final_prob_dict = {}
for logical_name in ['IL', 'XL', 'ZL', 'YL']:
    logical_string = [key for key, value in commutation_dict.items() if value == [logical_name]]
    print(f"{logical_name}_strings = {logical_string}")
    final_prob = sum(compute_probability(string) for string in logical_string)
    final_prob = sp.simplify(final_prob)
    final_prob_dict[logical_name] = final_prob
    print(f'final_prob for {logical_name}_strings: {final_prob}')



# Save final_prob_dict to a file
with open(f'repetition_code_prob_dict__n_{n}.pkl', 'wb') as file:
    pickle.dump(final_prob_dict, file)


stabilizers: ['XXII', 'IXXI', 'IIXX']
logical_operators: {'IL': 'IIII', 'XL': 'XIII', 'ZL': 'ZZZZ', 'YL': 'YZZZ'}
IL_strings = ['IIII', 'XXII', 'XIXI', 'IXXI', 'XIIX', 'IXIX', 'IIXX', 'XXXX']
final_prob for IL_strings: pI**4 + 6*pI**2*pX**2 + pX**4
XL_strings = ['XIII', 'IXII', 'IIXI', 'XXXI', 'IIIX', 'XXIX', 'XIXX', 'IXXX']
final_prob for XL_strings: 4*pI*pX*(pI**2 + pX**2)
ZL_strings = ['YYYY', 'ZZYY', 'ZYZY', 'YZZY', 'ZYYZ', 'YZYZ', 'YYZZ', 'ZZZZ']
final_prob for ZL_strings: pY**4 + 6*pY**2*pZ**2 + pZ**4
YL_strings = ['ZYYY', 'YZYY', 'YYZY', 'ZZZY', 'YYYZ', 'ZZYZ', 'ZYZZ', 'YZZZ']
final_prob for YL_strings: 4*pY*pZ*(pY**2 + pZ**2)
